# Loading module

In [ ]:
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

# Try to extract data

In [ ]:
client_path = r'CAL00001 Raw ClientA-Run1-Client.csv'
lab_path = r'CAL00001 Raw ClientA-Run1-Lab.csv'

In [ ]:
df_client = pd.read_csv(client_path, encoding='mac_roman')
df_lab = pd.read_csv(lab_path, encoding='mac_roman')

In [ ]:
title_index = 0
Backgrounds_num = 0
Measurements_num = 0

for i in range(len(df_client)):
    # get where the data start
    if 'DATA' in df_client.iloc[i].tolist()[0]:
        title_index = i + 1
        break
        
    # get the Background number
    elif 'Backgrounds' in df_client.iloc[i].tolist()[0]:
        Backgrounds_num = int(df_client.iloc[i].tolist()[2])
        
    # get the measurement number
    elif 'Measurements' in df_client.iloc[i].tolist()[0]:
        Measurements_num = int(df_client.iloc[i].tolist()[2])

In [ ]:
# store the title
title = df_client.iloc[title_index].values.tolist()

In [ ]:
df_total = df_client[title_index + 1:]
df_total.columns = title
df_total.drop(['P(kPa)', 'Comment'],axis=1,inplace=True)

# change the column type to numeric
df_total = df_total.apply(pd.to_numeric, errors='ignore')

In [ ]:
# len(df_total.groupby(['Filter']).size())
df_total.groupby(['Filter']).size()

In [ ]:
# dataframe of the before xray data
df_before = df_total[:Backgrounds_num]

In [ ]:
assert(len(df_before) == Backgrounds_num)

In [ ]:
# dataframe of the beam data with open xray
df_data = df_total[Backgrounds_num:-Backgrounds_num]

# check the number of measurement for each beam
df_size = df_data.groupby(['Filter']).size().to_frame('count')

# get the equipment which has more than Measurements_num measurements
duplicate_equip = list(df_size[df_size['count'] > Measurements_num].index)

df_noDuplicate = df_data[~df_data.Filter.isin(duplicate_equip)]
df_duplicate = df_data[df_data.Filter.isin(duplicate_equip)]
df_first = df_duplicate[:Measurements_num*3]
df_last = df_duplicate[-Measurements_num*3:]

In [ ]:
assert(len(df_first) + len(df_last) + len(df_noDuplicate) == len(df_data))

In [ ]:
# dataframe of the after xray data
df_after = df_total[-Backgrounds_num:]

In [ ]:
assert(len(df_after) == Backgrounds_num)

In [ ]:
# average the data
df_before_mean = df_before.groupby(['Filter']).agg({'mean'})
df_noDuplicate_mean = df_noDuplicate.groupby(['Filter']).agg({'mean'})
df_first_mean = df_first.groupby(['Filter']).agg({'mean'})
df_last_mean = df_last.groupby(['Filter']).agg({'mean'})
df_after_mean= df_after.groupby(['Filter']).agg({'mean'})

# Formal Calculation

In [ ]:
client_path = r'CAL00001 Raw ClientA-Run1-Client.csv'
lab_path = r'CAL00001 Raw ClientA-Run1-Lab.csv'

In [ ]:
class Processed_data():
    def __init__(self):
        self.df_before_mean = None
        self.df_mean = None
        self.df_after_mean = None

In [ ]:
def extraction(path):
    df = pd.read_csv(path, encoding='mac_roman')
    
    title_index = 0
    Backgrounds_num = 0
    Measurements_num = 0

    for i in range(len(df)):
        # get where the data start
        if 'DATA' in df.iloc[i].tolist()[0]:
            title_index = i + 1
            break

        # get the Background number
        elif 'Backgrounds' in df.iloc[i].tolist()[0]:
            Backgrounds_num = int(df.iloc[i].tolist()[2])

        # get the measurement number
        elif 'Measurements' in df.iloc[i].tolist()[0]:
            Measurements_num = int(df.iloc[i].tolist()[2])
            
    # store the title
    title = df.iloc[title_index].values.tolist()
    
    # extract all data
    df_total = df[title_index + 1:]
    df_total.columns = title
    df_total.drop(['kV','mA', 'HVLFilter(mm)', 'N', 'P(kPa)', 'Comment'], axis=1, inplace=True)

    # change the column type to numeric
    df_total = df_total.apply(pd.to_numeric, errors='ignore')
    
    
    # dataframe of the before xray data
    df_before = df_total[:Backgrounds_num]
    
    # dataframe of the beam data with open xray
    df_data = df_total[Backgrounds_num:-Backgrounds_num]

    # check the number of measurement for each beam
    df_size = df_data.groupby(['Filter']).size().to_frame('count')

    # seperate the beam which measure two times
    duplicate_beam = list(df_size[df_size['count'] > Measurements_num].index)

    df_noDuplicate = df_data[~df_data.Filter.isin(duplicate_beam)]
    df_duplicate = df_data[df_data.Filter.isin(duplicate_beam)]
    df_first = df_duplicate[:Measurements_num*len(duplicate_beam)]
    df_last = df_duplicate[-Measurements_num*len(duplicate_beam):]
    
    # dataframe of the after xray data
    df_after = df_total[-Backgrounds_num:]
    
    # average the data and store to object
    data = Processed_data()
    
    # put the filter which measure two times at the last
    data.df_mean = pd.concat([df_first.groupby(['Filter']).mean(), df_noDuplicate.groupby(['Filter']).mean(),
                           df_last.groupby(['Filter']).mean()], axis=0)
    
    data.df_before_mean = df_before.groupby(['Filter']).mean()
    data.df_after_mean= df_after.groupby(['Filter']).mean()
    
    # return number of duplicate_beam since kk need to know how many beams measure two times
    return data, len(duplicate_beam)

In [ ]:
# extract data
client_data, duplicate_num = extraction(client_path)
lab_data, _ = extraction(lab_path)

In [ ]:
# client calculation part
BgdIC1_Before = client_data.df_before_mean['Current1(pA)'].values[0]
BgdMC1_Before = client_data.df_before_mean['Current2(pA)'].values[0]

MC1 = (client_data.df_mean['Current1(pA)'] - BgdMC1_Before).to_frame()
IC1 = (client_data.df_mean['Current2(pA)'] - BgdIC1_Before).to_frame()
R1 = ((client_data.df_mean['Current1(pA)'] - BgdIC1_Before) / (client_data.df_mean['Current2(pA)'] - BgdMC1_Before)).to_frame()
TM1 = client_data.df_mean['T(MC)'].to_frame()
TA1 = client_data.df_mean['T(Air)'].to_frame()

In [ ]:
# lab calculation part
BgdIC2_Before = lab_data.df_before_mean['Current1(pA)'].values[0]
BgdMC2_Before = lab_data.df_before_mean['Current2(pA)'].values[0]

MC2 = (lab_data.df_mean['Current1(pA)'] - BgdMC2_Before).to_frame()
IC2 = (lab_data.df_mean['Current2(pA)'] - BgdMC2_Before).to_frame()
R2 = ((lab_data.df_mean['Current1(pA)'] - BgdIC2_Before) / (lab_data.df_mean['Current2(pA)'] - BgdMC2_Before)).to_frame()
TM2 = lab_data.df_mean['T(MC)'].to_frame()
TS2 = lab_data.df_mean['T(SC)'].to_frame()
H2 = lab_data.df_mean['H(%)'].to_frame()

# Contants part

In [ ]:
# read constant and KK from constant excel file
constant = r'E:\Unimelb\2021 S2\Software Project\test\constant\constant.xlsx'
df_constant = pd.read_excel(constant, sheet_name='constant')
df_KK = pd.read_excel(constant, sheet_name='Beams', usecols=[0,9])

In [ ]:
# get ma and WE
ma = df_constant['ma'].values[0]
WE = df_constant['WE'].values[0]

In [ ]:
# reform the KK into the same order as client_data.df_mean so that it will be easy to calculate directly 
# (do not need to extract the same index to do calculation. We can just calculate on data frame).

# get the filter of the first measurement from KK
df_KK = df_KK[df_KK.Filter.isin(client_data.df_mean.index)]
cats = client_data.df_mean.index[:-duplicate_num]
df_KK['Filter'] = pd.CategoricalIndex(df_KK['Filter'], ordered=True, categories=cats)
df_KK = df_KK.sort_values('Filter')

# get the filter which measure two times from KK
df_KK_dupkicate = df_KK[df_KK.Filter.isin(client_data.df_mean.index[-duplicate_num:])]
cats = client_data.df_mean.index[-duplicate_num:]
df_KK_dupkicate['Filter'] = pd.CategoricalIndex(df_KK_dupkicate['Filter'], ordered=True, categories=cats)
df_KK_dupkicate = df_KK_dupkicate.sort_values('Filter')

# concat together so that KK will have the same order as client_data.df_mean
df_KK = pd.concat([df_KK,df_KK_dupkicate], axis=0).set_index('Filter')

In [ ]:
# change the column type to numeric (other method)
# df_data['Current1(pA)'] = pd.to_numeric(df_data['Current1(pA)'])
# df_data['Current2(pA)'] = pd.to_numeric(df_data['Current2(pA)'])
# df_data['P(kPa)'] = pd.to_numeric(df_data['P(kPa)'])
# df_data['T(MC)'] = pd.to_numeric(df_data['T(MC)'])
# df_data['T(Air)'] = pd.to_numeric(df_data['T(Air)'])
# df_data['T(SC)'] = pd.to_numeric(df_data['T(SC)'])
# df_data['H(%)'] = pd.to_numeric(df_data['H(%)'])